In [13]:
import pandas as pd
import requests
import json
import re
import os
import time
from requests.exceptions import JSONDecodeError
from bs4 import BeautifulSoup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from serpapi import GoogleSearch

API_SECRET_KEY=...
BASE_URL=...
SERPAPI_KEY=...
os.environ["SERPAPI_API_KEY"]=SERPAPI_KEY
os.environ["OPENAI_API_KEY"]=API_SECRET_KEY
os.environ["OPENAI_API_BASE"]=BASE_URL

gene=pd.read_csv("file",sep='\t')

In [14]:
gene

,SampleName,SNP_Marker,SNP_Genotype,AlleleRatio,Type,TotalDepth,A,T,C,G,QC_Info
0,test,Popu.rs114514,AA,0,Homo,1919,1919,810,810,1,-
1,test,ID.rs268,AA,0,Homo,3570,3555,1,1,13,-
2,test,ID.rs5745448,TT,0,WT,782,0,780,2,0,-
3,test,ID.rs1049500,AA,0,Homo,1044,1037,0,0,7,-


In [15]:
recheck=[]
gene['IDs']=gene.iloc[:,1].str.extract(r"\.rs(.*)")
for id in gene['IDs']:
    try:
        response=requests.get(f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id={id}&rettype=json&retmode=text")
        if response.status_code == 200:
            data=json.dumps(response.json())
            print(f"Successfully searched rs{id}, checking...",end="")
            if ("clinical_significances" in data):
                print(f"wooh, rs{id} may have clinical significances")
                recheck.append([id,data])
            else:
                print(f"take it easy, it's all right")
        else:
            print(f"failed to find rs{id}")
    except JSONDecodeError :
        print(f'JSONDecodeError when searching rs{id}: no records were found or there\'s something wrong with the network')

JSONDecodeError when searching rs114514: no records were found or there's something wrong with the network
Successfully searched rs268, checking...wooh, rs268 may have clinical significances
Successfully searched rs5745448, checking...take it easy, it's all right
Successfully searched rs1049500, checking...wooh, rs1049500 may have clinical significances


In [16]:
recheck

[['268',
  '{"refsnp_id": "268", "create_date": "2000-09-19T17:02Z", "last_update_date": "2022-10-14T11:06Z", "last_update_build_id": "156", "dbsnp1_merges": [{"merged_rsid": "17850737", "revision": "126", "merge_date": "2006-03-11T06:16Z"}, {"merged_rsid": "52818902", "revision": "128", "merge_date": "2007-09-21T16:03Z"}, {"merged_rsid": "386571803", "revision": "136", "merge_date": "2014-08-6T12:19Z"}], "citations": [8541837, 17357073, 18280754, 18513389, 18660489, 18922999, 19041386, 19131662, 19489872, 19501493, 19602472, 20421590, 20429872, 20565774, 20650961, 21767357, 22024213, 22042884, 22239554, 22629316, 24319689, 25361584, 25474356, 25626708, 26975783, 30140409, 30333156, 32952508, 35351696], "lost_obs_movements": [], "present_obs_movements": [{"component_ids": [{"type": "subsnp", "value": "93851529"}, {"type": "subsnp", "value": "206439351"}, {"type": "subsnp", "value": "217321419"}, {"type": "subsnp", "value": "217397623"}, {"type": "subsnp", "value": "217418282"}, {"type"

In [17]:
print("rechecking using NCBI...")
for i in recheck:
    match=re.findall(r"clinical_significances\": \[([\"\w-]*)\]",i[1])
    dis=re.findall(r"disease_names\": \[([^\]]*)",i[1])
    i.append(match)
    i.append(dis)

rechecking using NCBI...


In [18]:
recheck

[['268',
  '{"refsnp_id": "268", "create_date": "2000-09-19T17:02Z", "last_update_date": "2022-10-14T11:06Z", "last_update_build_id": "156", "dbsnp1_merges": [{"merged_rsid": "17850737", "revision": "126", "merge_date": "2006-03-11T06:16Z"}, {"merged_rsid": "52818902", "revision": "128", "merge_date": "2007-09-21T16:03Z"}, {"merged_rsid": "386571803", "revision": "136", "merge_date": "2014-08-6T12:19Z"}], "citations": [8541837, 17357073, 18280754, 18513389, 18660489, 18922999, 19041386, 19131662, 19489872, 19501493, 19602472, 20421590, 20429872, 20565774, 20650961, 21767357, 22024213, 22042884, 22239554, 22629316, 24319689, 25361584, 25474356, 25626708, 26975783, 30140409, 30333156, 32952508, 35351696], "lost_obs_movements": [], "present_obs_movements": [{"component_ids": [{"type": "subsnp", "value": "93851529"}, {"type": "subsnp", "value": "206439351"}, {"type": "subsnp", "value": "217321419"}, {"type": "subsnp", "value": "217397623"}, {"type": "subsnp", "value": "217418282"}, {"type"

In [19]:
res=pd.DataFrame(recheck,columns=['id','data','clinical','dis'])
res['sit']=0
row=0
for i in res['clinical']:
    if len(i)==1:
        res.loc[row,'sit']=-10000
    else:
        sum=0
        for j in i:
            if ("benign" in j):
                sum+=-3
            elif ("uncertain-significance" in j):
                sum+=0
            elif ("risk-factor" in j):
                sum+=3
            elif ("conflicting-interpretations-of-pathogenicity" in j):
                sum+=3
            elif ("pathogenic" in j):
                sum+=5
        res.loc[row,'sit']=round(sum/len(i))
    row+=1

In [20]:
res

,id,data,clinical,dis,sit
0,268,"{""refsnp_id"": ""268"", ""create_date"": ""2000-09-1...","[""pathogenic"", ""risk-factor"", ""uncertain-signi...","[""Hyperlipidemia, familial combined, LPL relat...",2
1,1049500,"{""refsnp_id"": ""1049500"", ""create_date"": ""2000-...","[""benign""]","[""not provided""]",-10000


In [21]:
for i in range(len(res['sit'])):
    if len(res.loc[i,'clinical'])==1:
        print("Few records were found about this gene, ",end="")
        if ('"benign"' in res.loc[i,'clinical']) :
            print("but they are good ones, congrats!")
        elif ('"uncertain-significance"' in res.loc[i,'clinical']):
            print("and whether this do harm or do good still remains to be unknown")
        elif ('"risk-factor"' in res.loc[i,'clinical']):
            print("but existing records show that this might lead to higher risks of some diseases, so take care!")
        elif ('"conflicting-interpretations-of-pathogenicity"' in res.loc[i,'clinical']):
            print("scientists are still debating on the impact of it")
        elif ('"pathogenic"' in res.loc[i,'clinical']):
            print("but there is evidence that this might lead to some suffer")
    else:
        print("There are multiple records about it and its impact vary, ",end="")
        if (res.loc[i,'sit'])<=0:
            print("but generally they seem to do more good than bad")
        elif (res.loc[i,'sit'])<2:
            print("Some say it's harmless and some say it's not, we still don't know for sure yet")
        else: 
            print("and past records show that it might br pathogenic to some degree")

There are multiple records about it and its impact vary, and past records show that it might br pathogenic to some degree
Few records were found about this gene, but they are good ones, congrats!


In [22]:
def split(data):
    index=data.find("[")
    return data[:index],data[index:]

def process_html(text):
    soup=BeautifulSoup(text,'html.parser')
    p_tags = soup.find_all('p')
    data="".join([p_tag.get_text().strip() for p_tag in p_tags])
    data=re.sub(r"Have questions\? Visit https://www.reddit.com/r/SNPediars\d*, ","",data)   
    return data

def GPT_summarize_1(input_con):
    llm=OpenAI(temperature=0)
    prompt=PromptTemplate(
        input_variables=['cases'],
        template="Below are reports of an SNP, summarize them, note that every [PMID] is one separate case,{cases}, don't summarize them one by one, please summarize them generally, do not contain words like [PMID]"
    )
    chain=LLMChain(llm=llm, prompt=prompt)
    print(chain.invoke(input_con)['text']) 

def GPT_extract(res,j,data2):
    llm=OpenAI(temperature=0)
    prompt=PromptTemplate(
            input_variables=['dis','dt2'],
            template="{dis},{dt2},The messages above contains the disease information of a certain SNP, please find what diseases are mentioned, return only disease names, don't care about subtypes, and skip it if it was words like not provided or not specified, give only the two most frequent ones"
    )
    input_data={
            'dis':res.loc[j,'dis'],
            'dt2':data2
    }
    chain=LLMChain(llm=llm,prompt=prompt)
    dis_to_be_searched=chain.invoke(input_data)['text']
    return dis_to_be_searched

def GPT_summarize_2(dis_to_be_searched):
    llm=OpenAI(temperature=0)
    tools=load_tools(['serpapi'],llm=llm)
    prompt=PromptTemplate(
        input_variables=['dis'],
        template="{dis} search these two diseases, give some simple explanation to their pathology and prevention advice"
    )
    input_prompt=prompt.format(dis=dis_to_be_searched)
    agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=False)
    print(agent.run(input_prompt))

In [24]:
for i in res['id']:
    j=0
    ans=requests.get(f"https://www.snpedia.com/index.php/rs{i}")
    if (ans.status_code!=200):
        print(f"No more information about rs{i} were found")
    else:
        data=process_html(ans.text)
        data1,data2=split(data)
        print(f"There is more information about rs{i} found on https://www.snpedia.com/index.php/rs{i} summarizing...")
        print(f"rs{i},", data1) 
        GPT_summarize_1(data2)
        dis_to_be_searched=GPT_extract(res,j,data2)
        GPT_summarize_2(dis_to_be_searched)
    j+=1

There is more information about rs268 found on https://www.snpedia.com/index.php/rs268 summarizing...
rs268, also known as LPL Asn291Ser as well as LPL N291S or N318S, a SNP in the lipoprotein lipase LPL gene, has been linked to increased susceptibility to hypertriglyceridemia, heart disease, Type-2 diabetes, idiopathic venous thromboembolism. A systematic review and meta-analysis of the relationship between lipoprotein lipase Asn291Ser variant and diseases. The summary standardized mean difference (SMD) of plasma triglyceride (TG) for carriers compared with noncarriers of the Asn291Ser variant was 3.23 (P < 0.00001). The summary SMD of plasma HDL-cholsterol (HDL-C) for carriers compared with noncarriers of the Asn291Ser variant was -3.42 (P < 0.0001). The summary SMD of the association of the Asn291Ser variant with plasma TG increased with increasing age and weight gain. Significant interactions between the LPL Asn291Ser variant and fasting glucose, T2DM, and CHD (Coronary Heart Disea